In [1]:
from nilmtk import DataSet
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
# Load dataset (change path if needed)
dataset = DataSet("D:/lusip/ukdale.h5/ukdale.h5")


In [3]:
elec = dataset.buildings[1].elec

# Load active power only
mains_df = next(elec.mains().load(sample_period=6))[('power', 'active')].to_frame()
mains_df.columns = ['mains']

fridge_df = next(elec['fridge'].load(sample_period=6))[('power', 'active')].to_frame()
fridge_df.columns = ['fridge']

# Align timestamps
aligned_df = mains_df.join(fridge_df, how='inner')
aligned_df.dropna(inplace=True)


c:\Users\tanmay\AppData\Local\Programs\Python\Python313\Lib\site-packages\nilmtk\utils.py:509: UserWarning: Found duplicate index. Keeping first value
  warnings.warn("Found duplicate index. Keeping first value")
c:\Users\tanmay\AppData\Local\Programs\Python\Python313\Lib\site-packages\nilmtk\utils.py:501: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  return fill_method(how(data.resample(rule, **resample_kwargs)))
c:\Users\tanmay\AppData\Local\Programs\Python\Python313\Lib\site-packages\nilmtk\utils.py:501: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  return fill_method(how(data.resample(rule, **resample_kwargs)))


In [4]:
mains_scaler = MinMaxScaler()
fridge_scaler = MinMaxScaler()

aligned_df['mains'] = mains_scaler.fit_transform(aligned_df[['mains']])
aligned_df['fridge'] = fridge_scaler.fit_transform(aligned_df[['fridge']])


In [5]:
def create_sequences(df, window_size):
    X, y = [], []
    for i in range(len(df) - window_size):
        X.append(df['mains'].values[i:i + window_size])
        y.append(df['fridge'].values[i + window_size // 2])
    return np.array(X), np.array(y)

WINDOW_SIZE = 99
X, y = create_sequences(aligned_df, WINDOW_SIZE)


In [6]:
X = X.reshape(-1, WINDOW_SIZE, 1)  # Shape: (samples, 99, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error
import optuna
import numpy as np

# 🔧 Define your model class
class LSTMTransformer(nn.Module):
    def __init__(self, hidden_size, num_heads, ff_dim, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size, batch_first=True)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.transformer(x)
        center = x[:, x.size(1) // 2, :]
        return self.fc(center).squeeze()

# 🎯 Define Optuna objective
def objective(trial):
    # Sample hyperparameters
    num_heads = trial.suggest_categorical('num_heads', [2, 4, 8])
    hidden_size_choices = [h for h in range(32, 129, 8) if h % num_heads == 0]
    hidden_size = trial.suggest_categorical('hidden_size', hidden_size_choices)
    ff_dim = trial.suggest_int('ff_dim', 64, 256)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])

    # ⚡ Use subset for faster training
    subset_size = 300_000
    X_sub = X_train[:subset_size]
    y_sub = y_train[:subset_size]

    train_dataset = TensorDataset(
        torch.tensor(X_sub, dtype=torch.float32),
        torch.tensor(y_sub, dtype=torch.float32)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Build model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LSTMTransformer(hidden_size, num_heads, ff_dim, num_layers).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 🔁 Train for 5 epochs
    model.train()
    for epoch in range(10):
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            output = model(xb)
            loss = criterion(output, yb)
            loss.backward()
            optimizer.step()

    # 📈 Evaluate on test set
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for xb, yb in DataLoader(
            TensorDataset(
                torch.tensor(X_test, dtype=torch.float32),
                torch.tensor(y_test, dtype=torch.float32)
            ),
            batch_size=batch_size
        ):
            xb = xb.to(device)
            preds = model(xb).cpu().numpy()
            predictions.extend(preds)
            actuals.extend(yb.numpy())

    # Inverse scale
    preds_inv = fridge_scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
    actuals_inv = fridge_scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten()

    # Return MAE to minimize
    return mean_absolute_error(actuals_inv, preds_inv)

c:\Users\tanmay\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

print("Best MAE:", study.best_value)
print("Best Hyperparameters:", study.best_params)


[I 2025-07-03 23:04:32,935] A new study created in memory with name: no-name-7f63a812-afeb-4493-989c-d7c2169f4042
[I 2025-07-03 23:51:44,833] Trial 0 finished with value: 31.293428421020508 and parameters: {'num_heads': 4, 'hidden_size': 128, 'ff_dim': 140, 'num_layers': 1, 'lr': 0.0010223717867684189, 'batch_size': 128}. Best is trial 0 with value: 31.293428421020508.
[I 2025-07-04 01:29:36,041] Trial 1 finished with value: 26.998451232910156 and parameters: {'num_heads': 8, 'hidden_size': 72, 'ff_dim': 208, 'num_layers': 2, 'lr': 0.00014062462052452567, 'batch_size': 128}. Best is trial 1 with value: 26.998451232910156.
[I 2025-07-04 09:15:08,025] Trial 2 finished with value: 45.341796875 and parameters: {'num_heads': 8, 'hidden_size': 72, 'ff_dim': 231, 'num_layers': 3, 'lr': 0.0017650216000706523, 'batch_size': 64}. Best is trial 1 with value: 26.998451232910156.
[I 2025-07-04 11:06:15,931] Trial 3 finished with value: 23.254003524780273 and parameters: {'num_heads': 4, 'hidden_siz

Best MAE: 23.254003524780273
Best Hyperparameters: {'num_heads': 4, 'hidden_size': 96, 'ff_dim': 247, 'num_layers': 3, 'lr': 0.00018107399357726567, 'batch_size': 128}
